<a href="https://colab.research.google.com/github/ashleykim00/ESAA/blob/main/0927_%EB%8D%B0%EC%9D%B4%EC%BD%98_%ED%95%84%EC%82%AC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# [NLP 언제까지 미룰래? 일단 들어와!!] #4. word embedding

# 1. Keras Embedding Layer

In [2]:
from google.colab import drive

drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [4]:
import pandas as pd
path='/content/gdrive/MyDrive/train.csv'
train = pd.read_csv(path)

In [5]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

def text2sequence(train_text, max_len=100):
    
    tokenizer = Tokenizer() #keras의 vectorizing 함수 호출
    tokenizer.fit_on_texts(train_text) #train 문장에 fit
    train_X_seq = tokenizer.texts_to_sequences(train_text) #각 토큰들에 정수 부여
    vocab_size = len(tokenizer.word_index) + 1 #모델에 알려줄 vocabulary의 크기 계산
    print('vocab_size : ', vocab_size)
    X_train = pad_sequences(train_X_seq, maxlen = max_len) #설정한 문장의 최대 길이만큼 padding
    
    return X_train, vocab_size, tokenizer

train_X, vocab_size, vectorizer = text2sequence(train['text'], max_len = 100)

vocab_size :  42331


In [6]:
from keras.models import Sequential

In [10]:
from tensorflow.keras.preprocessing.text import Tokenizer
from keras.layers import Dense,Activation,Embedding
import numpy as np

In [12]:
model = Sequential()
model.add(Embedding(vocab_size, 128, input_length = 100))

# word2vec

In [13]:
import gensim
word2vec = gensim.models.KeyedVectors.load_word2vec_format('/content/gdrive/MyDrive/GoogleNews-vectors-negative300.bin.gz', binary = True)

In [21]:
tokenizer = Tokenizer() #keras의 vectorizing 함수 호출

In [20]:
embedding_matrix = np.zeros((vocab_size, 300)) #300차원의 임베딩 매트릭스 생성

for index, word in enumerate(tokenizer.word_index): #vocabulary에 있는 토큰들을 하나씩 넘겨줍니다.
    if word in word2vec: #넘겨 받은 토큰이 word2vec에 존재하면(이미 훈련이 된 토큰이라는 뜻)
        embedding_vector = word2vec[word] #해당 토큰에 해당하는 vector를 불러오고
        embedding_mxtrix[i] = embedding_vector #해당 위치의 embedding_mxtrix에 저장합니다.
    else:
        print("word2vec에 없는 단어입니다.")
        break

In [23]:
model = Sequential()
model.add(Embedding(vocab_size, 300,weights = [embedding_matrix], input_length = 100))

# glove

In [25]:
2. # load the whole embedding into memory
glove = dict()
f = open('/content/gdrive/MyDrive/word-embeddings/glove/glove.txt')
for line in f:
    values = line.split()
    word = values[0]
    vector = np.asarray(values[1:], dtype='float32')
    glove[word] = vector
f.close()

In [26]:
embedding_matrix = np.zeros((vocab_size, 300)) #300차원의 임베딩 매트릭스 생성

for index, word in enumerate(tokenizer.word_index): #vocabulary에 있는 토큰들을 하나씩 넘겨줍니다.
    if word in glove: #넘겨 받은 토큰이 word2vec에 존재하면(이미 훈련이 된 토큰이라는 뜻)
        embedding_vector = glove[word] #해당 토큰에 해당하는 vector를 불러오고
        embedding_mxtrix[i] = embedding_vector #해당 위치의 embedding_mxtrix에 저장합니다.
    else:
        print("glove 없는 단어입니다.")
        break

In [28]:
model = Sequential()
model.add(Embedding(vocab_size, 300,weights = [embedding_matrix], input_length = 100))

# Fasttext

In [41]:
from gensim.models.keyedvectors import KeyedVectors
FastText = gensim.models.KeyedVectors.load_word2vec_format('/content/gdrive/MyDrive/word-embeddings/fasttext/fasttext.vec', binary=True, unicode_errors='ignore')

In [42]:
embedding_matrix = np.zeros((vocab_size, 300)) #300차원의 임베딩 매트릭스 생성

for index, word in enumerate(tokenizer.word_index): #vocabulary에 있는 토큰들을 하나씩 넘겨줍니다.
    if word in word2vec: #넘겨 받은 토큰이 word2vec에 존재하면(이미 훈련이 된 토큰이라는 뜻)
        embedding_vector = word2vec[word] #해당 토큰에 해당하는 vector를 불러오고
        embedding_mxtrix[i] = embedding_vector #해당 위치의 embedding_mxtrix에 저장합니다.

In [44]:
model = Sequential()
model.add(Embedding(vocab_size, 300,weights = [embedding_matrix], input_length = 100))

# [NLP 언제까지 미룰래? 일단 들어와!!] #5. Modeling(완)

In [48]:
!pip3 install konlpy

     |████████████████████████████████| 19.4 MB 2.0 MB/s 
     |████████████████████████████████| 86 kB 5.4 MB/s 
     |████████████████████████████████| 448 kB 56.6 MB/s 
  Attempting uninstall: beautifulsoup4
    Found existing installation: beautifulsoup4 4.6.3
    Uninstalling beautifulsoup4-4.6.3:
      Successfully uninstalled beautifulsoup4-4.6.3


In [61]:
from konlpy.tag import Okt
import re
import tqdm 

def text_preprocessing(text_list):
    
    stopwords = ['을', '를', '이', '가', '은', '는', 'null'] #불용어 설정
    tokenizer = Okt() #형태소 분석기 
    token_list = []
    
    for text in tqdm.tqdm(text_list):
        txt = re.sub('[^가-힣a-z]', ' ', text) #한글과 영어 소문자만 남기고 다른 글자 모두 제거
        token = tokenizer.morphs(txt) #형태소 분석
        token = [t for t in token if t not in stopwords or type(t) != float] #형태소 분석 결과 중 stopwords에 해당하지 않는 것만 추출
        token_list.append(token)
        
    return token_list, tokenizer

train['token'], okt = text_preprocessing(train['text'])

100%|██████████| 54879/54879 [08:57<00:00, 102.13it/s]


In [54]:
train.head()

,index,text,author,token
0,0,"He was almost choking. There was so much, so m...",3,"[e, was, almost, choking, here, was, so, much,..."
1,1,"“Your sister asked for it, I suppose?”",2,"[our, sister, asked, for, it, suppose]"
2,2,"She was engaged one day as she walked, in per...",1,"[he, was, engaged, one, day, as, she, walked, ..."
3,3,"The captain was in the porch, keeping himself ...",4,"[he, captain, was, in, the, porch, keeping, hi..."
4,4,"“Have mercy, gentlemen!” odin flung up his han...",3,"[ave, mercy, gentlemen, odin, flung, up, his, ..."


In [55]:
def text2sequence(train_text, max_len=1000):
    
    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(train_text)
    train_X_seq = tokenizer.texts_to_sequences(train_text)
    vocab_size = len(tokenizer.word_index) + 1
    print('vocab_size : ', vocab_size)
    X_train = pad_sequences(train_X_seq, maxlen = max_len)
    return X_train, vocab_size, tokenizer

train_y = train['author']
train_X, vocab_size, vectorizer = text2sequence(train['token'], max_len = 100)
print(train_X.shape, train_y.shape)

vocab_size :  36342
(54879, 100) (54879,)


In [59]:
word2vec = gensim.models.KeyedVectors.load_word2vec_format('/content/gdrive/MyDrive/GoogleNews-vectors-negative300.bin.gz', binary = True)
embedding_matrix = np.zeros((vocab_size, 300))

for index, word in enumerate(tokenizer.word_index):
    if word in word2vec :
        embedding_vector = word2vec[word] 
        embedding_mxtrix[i] = embedding_vector 
    else:
        print("word2vec에 없는 단어입니다.")
        break

In [60]:
def LSTM(vocab_size, max_len=1000):
    model = Sequential()
    model.add(Embedding(vocab_size, 300,weights = [embedding_matrx], input_length = max_len)) #임베딩 가중치 적용 코드
    model.add(SpatialDropout1D(0.3))
    model.add(LSTM(64))
    model.add(Dropout(0.5))
    model.add(Dense(64, activation='relu', kernel_regularizer = regularizers.l2(0.001)))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics='accuracy')
    model.summary()
    return model